# Grab Challenge - Traffic Management

This notebook preprocesses the time and spatial features of traffic demand dataset in order to construct XGBoost Regressor Model.

In [1]:
import geohash
import numpy as np
import pandas as pd
import random
import xgboost
import pickle

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from annoy import AnnoyIndex

## Data Preprocessing

#### Load Train Dataset

In [2]:
df = pd.read_csv("data/training.csv")

#### Split timestamp to hour and minute

In [3]:
df['hour'] = df['timestamp'].apply(lambda x: int(x.split(':')[0]))
df['minute'] = df['timestamp'].apply(lambda x: int(x.split(':')[1]))

#### Convert geohash to latitude and longitude

In [4]:
def geohash2LatLong(gh):
    lat_long = geohash.decode_exactly(gh)
    return float(lat_long[0]), float(lat_long[1])

lats = []
longs = []
for index, row in df.iterrows():
    lat, long = geohash2LatLong(row['geohash6'])
    lats.append(lat)
    longs.append(long)

df['lat'] = lats
df['long'] = longs

### Additional Feature 1: Past Days Demand Value

This feature looks up the previous demand values of each geohash location for past few days. This is to incorporate time-series feature of the dataset into the prediction.

In [5]:
# Number of past days to look up as feature
no_days_lookup = 10

In [6]:
day_mean = df.groupby(['geohash6', 'day'])['demand'].mean()

In [7]:
# Insert additional feature columns
for i in range(no_days_lookup):
    column = 'D-' + str(i+1) + ' mean'
    df.insert(len(df.columns), column, value=0.0)

In [8]:
for i, row in df.iterrows():
#     print(i)
    geohash_value = row['geohash6']
    day = row['day']
    for j in range(no_days_lookup):
        column = 'D-' + str(j+1) + ' mean'
        insert_tuple = (geohash_value, day-(j+1))
        if insert_tuple in day_mean.index:
            df.set_value(i, column, float(day_mean.loc[[insert_tuple]]))
        else:
            df.set_value(i, column, 0.0)

C:\Users\Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


### Additional Feature 2: Nearest Neighbours Demand Value

This feature uses Spotify's Annoy library to find the nearest neighbours of each data entries at given timeframe, and uses product of neighbours demand value and distance to the neigbhour as the additional feature to incorporate spatial element to the prediction.

In [9]:
# Number of closest neighbours to use as feature
no_neighbours = 5

In [10]:
# Sort dataframe to find nearest neighbours in each day and time
df = df.sort_values(by=['day', 'timestamp'])

In [11]:
# Insert additional feature columns
for i in range(no_neighbours):
    column = str(i+1) + '_neighbour'
    df.insert(len(df.columns), column, value=0.0)

In [12]:
# Data in same timeframe (date and time) belongs to the same 'window'

windows = []
window_indices = []
day = None
timestamp = None
for i, row in df.iterrows():
    if day == None and timestamp == None:
        day = row['day']
        timestamp = row['timestamp']
    elif row['day'] != day or row['timestamp'] != timestamp:
        windows.append(window_indices)
        window_indices = []
        day = None
        timestamp = None
    window_indices.append(i)

In [13]:
# Insert (demand * distance) value of nearest neighbours into dataframe 

for window in windows:
    t = AnnoyIndex(2, metric='euclidean')
    for index in window:
        t.add_item(index, [df['lat'][index], df['long'][index]])
    t.build(10)
    
    for index in window:
        indices, distances = t.get_nns_by_item(i=index, n=no_neighbours+1, include_distances=True)
        neighbours = list(zip(indices, distances))[1:] # Exclusde first neighbour => itself
        for i, neighbour in enumerate(neighbours):
            neighbour_index = neighbour[0]
            neighbour_distance = neighbour[1]
            neighbour_demand = df['demand'][neighbour_index]
            column = str(i+1) + '_neighbour'
            df[column][index] = neighbour_distance * neighbour_demand

C:\Users\Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
# Reshuffle dataframe
df = df.sample(frac=1)

### Train XGBoost Regressor Model

In [15]:
y = df['demand']
X = df.drop(['geohash6', 'timestamp', 'demand'], axis=1)

In [16]:
model = xgboost.XGBRegressor(
    colsample_bytree=0.4,
    gamma=0,                 
    learning_rate=0.07,
    max_depth=3,
    min_child_weight=1.5,
    n_estimators=1000,
    verbosity=2,
    reg_alpha=0.75,
    reg_lambda=0.45,
    subsample=0.6,
    seed=42)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)

model.fit(X_train, y_train, verbose=True)

preds = model.predict(X_test)

C:\Users\Han\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[17:46:17] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:46:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:46:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[17:46:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:46:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[17:46:17] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[17:46:1

[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:

[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:

[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:

[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:

[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:46:18] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:

[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:

[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:46:19] INFO: C:\Jenkins\workspace\xgboost-win64_release_0.90\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[17:

In [19]:
# Calculate Root Mean Square Error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.138969


In [20]:
# Check Feature Importance Values
print(list(zip(list(X_train.columns), model.feature_importances_)))

[('day', 0.049867738), ('hour', 0.06098895), ('minute', 0.035134714), ('lat', 0.094535716), ('long', 0.0915482), ('D-1 mean', 0.1314968), ('D-2 mean', 0.1610843), ('D-3 mean', 0.1354391), ('1_neighbour', 0.059878793), ('2_neighbour', 0.040308543), ('3_neighbour', 0.034843586), ('4_neighbour', 0.05222229), ('5_neighbour', 0.052651297)]


In [21]:
# Save xgboost model to file
pickle.dump(model, open("traffic_xgboost.dat", "wb"))